### Saved Model - Format

In [ ]:
#!pip install pyyaml h5py

In [18]:
import os
import sys
import yaml
import tensorflow as tf
from tensorflow import keras

In [38]:
from model.dataset import Dataset
from model.fp.nnfp import get_fingerprinter
from model.fp.NTxent_loss_single_gpu import NTxentLoss

In [39]:
def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

In [41]:
config:str = "default" 
cfg = load_config(config)

cli: Configuration from ./config/default.yaml


In [42]:
if cfg['LOSS']['LOSS_MODE'].upper() == 'NTXENT': # Default
    loss_obj_train = NTxentLoss(
        n_org=cfg['BSZ']['TR_N_ANCHOR'],
        n_rep=cfg['BSZ']['TR_BATCH_SZ'] - cfg['BSZ']['TR_N_ANCHOR'],
        tau=cfg['LOSS']['TAU'])

In [44]:
checkpoint_dir = '/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/'
#os.listdir(checkpoint_dir)

In [45]:
latest = tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/')

In [46]:
model = get_fingerprinter(cfg, trainable=True)

In [47]:
dataset = Dataset(cfg)
train_ds = dataset.get_train_ds(cfg['DATA_SEL']['REDUCE_ITEMS_P'])

In [51]:
model.compile(optimizer='ADAM', loss=loss_obj_train, metrics=['accuracy', 'loss_weights='])

In [ ]:
model.fit(train_ds, ..., epochs=5)

In [54]:
from tensorflow.keras.models import save_model

model.save('my_model.keras')

/home/rodrigo/anaconda3/envs/tf/lib/python3.11/site-packages/keras/src/saving/saving_api.py:164: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  saving_lib.save_model(model, filepath)


TypeError: Cannot serialize object <model.fp.NTxent_loss_single_gpu.NTxentLoss object at 0x7bab001e91d0> of type <class 'model.fp.NTxent_loss_single_gpu.NTxentLoss'>. To be serializable, a class must implement the `get_config()` method.

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model.load_weights(latest)